In [1]:
# data cleanning and web scrapping for upcoming moveis from TMDB


In [6]:
# load the useful library
import requests
import pandas as pd
from bs4 import BeautifulSoup


In [7]:
from IPython import display

In [103]:
needed_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}
response = requests.get("https://www.themoviedb.org/movie", headers = needed_headers )
page_contents = response.text
page_contents[:500]
doc= BeautifulSoup(page_contents, 'html.parser')
response.status_code


200

In [106]:
# load the movie pages using request
#remember to create a personal API so that we can using the request
def get_movies_page(movies_url):
    """
    Function to download a web page using `requests` and check the status code to validate
    if the call was successful. 
    """
    #  using `requests`
    needed_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}
    response = requests.get(movies_url, headers = needed_headers )
    # Check  was successful
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(movies_url))
    # Parse the `response' text using BeautifulSoup
    movies_doc = BeautifulSoup(response.text, 'html.parser')
    return movies_doc

In [129]:
# get the movies links for those movies and save it from scrapping the further info:
def get_url(movies_names_tags):
    links = []
    for movie_links in movies_names_tags:
        links.append(movie_links.a['href'])
    print(links)

In [136]:
# get 20 pages movies url and info:
def movie_scrap():
    movies_url_list = [] # store the url in dictionary that we can perform furhter operations
    movies_names = []
    for i in range(1,10):
        movies_url = "https://www.themoviedb.org/movie?page=%d" %(i)
        movies_url_list.append(get_movies_page(movies_url))
    
    movies_names_tags = movies_url_list.find_all('h2')[4:]  #Exclude the first 4 lines
    url = get_url(movies_names_tags)
    for h2 in movies_names_tags:
        names.append(h2.a.text.strip())
    return url

movie_scrap()


AttributeError: 'list' object has no attribute 'find_all'

In [133]:
# we would like to get moveis name from the h2 content:
movies_names_tags = doc.find_all('h2')[4:]  #Exclude the first 4 lines
names = []
for h2 in movies_names_tags:
    names.append(h2.a.text.strip())
print(names)

['Black Adam', 'R.I.P.D. 2: Rise of the Damned', 'Paradise City', 'Hex', 'Disenchanted', 'Black Panther: Wakanda Forever', 'Emily the Criminal', 'Lost Bullet 2', 'Sniper: The White Raven', 'Medieval', 'On the Line', 'Smile', 'Frank and Penelope', 'Margaux', 'Corrective Measures', 'The Soccer Football Movie', "Tom and Jerry Snowman's Land", 'Slumberland', 'Enola Holmes 2', 'The Woman King']


In [83]:
# cancate the movie and the movie url:
def get_movies_names(doc):
    """
    Function to extract the movie names from HTML source code using BeautifulSoup.
    """
    movies_names_tags = doc.find_all('h2')[4:]  #Exclude the first 4 lines
    movies_names = []
    # Loop through the page get all the movie names from the page
    for h2 in movies_names_tags:
        movies_names.append(h2.a.text.strip())
    return movies_names

In [ ]:
def movies():
    

In [ ]:
import os
base_link = "https://www.themoviedb.org/tv"

# 'i' here means the number of page we want to extract


In [84]:
get_movies_names(doc)

['Black Adam',
 'R.I.P.D. 2: Rise of the Damned',
 'Paradise City',
 'Hex',
 'Disenchanted',
 'Black Panther: Wakanda Forever',
 'Emily the Criminal',
 'Lost Bullet 2',
 'Sniper: The White Raven',
 'Medieval',
 'On the Line',
 'Smile',
 'Frank and Penelope',
 'Margaux',
 'Corrective Measures',
 'The Soccer Football Movie',
 "Tom and Jerry Snowman's Land",
 'Slumberland',
 'Enola Holmes 2',
 'The Woman King']

In [85]:
def get_movies_urls(doc):
    """
    Function to extract the movie links from HTML source code using BeautifulSoup. 
    """
    movies_urls = []
    base_url = 'https://www.themoviedb.org'
    movies_names_tags = doc.find_all('h2')[4:]  #Exclude the first 4 lines
    # Loop through the webpage to get the URL of each movie
    for tag in movies_names_tags:
        movies_urls.append(base_url + tag.a['href'])
    return movies_urls

In [86]:
get_movies_urls(doc)

['https://www.themoviedb.org/movie/436270',
 'https://www.themoviedb.org/movie/1013860',
 'https://www.themoviedb.org/movie/829799',
 'https://www.themoviedb.org/movie/988233',
 'https://www.themoviedb.org/movie/338958',
 'https://www.themoviedb.org/movie/505642',
 'https://www.themoviedb.org/movie/862965',
 'https://www.themoviedb.org/movie/948276',
 'https://www.themoviedb.org/movie/966220',
 'https://www.themoviedb.org/movie/551271',
 'https://www.themoviedb.org/movie/979924',
 'https://www.themoviedb.org/movie/882598',
 'https://www.themoviedb.org/movie/899294',
 'https://www.themoviedb.org/movie/846778',
 'https://www.themoviedb.org/movie/872177',
 'https://www.themoviedb.org/movie/1037858',
 'https://www.themoviedb.org/movie/1018403',
 'https://www.themoviedb.org/movie/668461',
 'https://www.themoviedb.org/movie/829280',
 'https://www.themoviedb.org/movie/724495']

In [87]:
def get_detailed_movie_page(movies_url):
    """
    Function to read the HTML source code using BeautifulSoup.
    """
    # Download the page
    response = requests.get(movies_url)
    # Check successful response
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(movies_url))
    # Parse using Beautiful soup
    movies_doc = BeautifulSoup(response.text, 'html.parser')
    return movies_doc

In [98]:
doc1 = get_movies_page('https://www.themoviedb.org/movie/436270')

200


In [99]:
# Find the `div` tag under `facts` class to get the release date, genre and runtime 
div_tags = doc1.find('div', class_ = 'facts')

release_date = div_tags.text.split()[1]
genre = div_tags.text.split()[3:-2]
runtime = div_tags.text.split()[-2:]

# Print and validate the result is correct
print(release_date, genre, runtime)

10/21/2022 ['Action,', 'Fantasy,', 'Science', 'Fiction'] ['2h', '5m']


In [100]:
d_tags = doc1.find_all('div', {'class':'scroller_wrap should_fade is_fading'})

# Print and validate the result
print (d_tags[0].text.strip().partition("\n")[0])

Dwayne Johnson


In [193]:
def get_movies_info(doc):
    """
    Function to get the movie informations - 
    release date, genre, runtime and director.
    """
    div1_tags = doc.find('div', class_ = 'facts')
    release_date = div1_tags.text.split()[1]
    genre = div1_tags.text.split()[3:-2]
    runtime = div1_tags.text.split()[-2:]
    
    div2_tags = doc.find_all('div', {'class':'scroller_wrap should_fade is_fading'})
    director = "None" #div2_tags[0].text.strip().partition("\n")[0]
    
    return release_date, genre, runtime, director

In [194]:
get_movies_info(doc1) 

('11/15/2022',
 ['Fantasy,', 'Action,', 'Comedy,', 'Crime'],
 ['1h', '42m'],
 'None')

## let's start new here:

In [195]:
## let's start new here:


In [196]:
def get_movies_page(tmdb_url):

    needed_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}
    response = requests.get(tmdb_url, headers = needed_headers )
    # Access the webpage using `requests`
    # Check if the request was successful
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(tmdb_url))
    # Parse the `response' text using BeautifulSoup
    movies_doc = BeautifulSoup(response.text, 'html.parser')
    return movies_doc

#get_movies_page("https://www.themoviedb.org/movie?page=%d" %(i))
doc = get_movies_page("https://www.themoviedb.org/movie")


In [197]:
def get_movies_names(doc):

    movies_names_tags = doc.find_all('h2')[4:]  #Exclude the first 4 lines
    movies_names = []
    # Loop through the page get all the movie names from the page
    for h2 in movies_names_tags:
        movies_names.append(h2.a.text.strip())
    return movies_names
get_movies_names(doc)

['Black Adam',
 'R.I.P.D. 2: Rise of the Damned',
 'Paradise City',
 'Hex',
 'Disenchanted',
 'Black Panther: Wakanda Forever',
 'Emily the Criminal',
 'Lost Bullet 2',
 'Sniper: The White Raven',
 'Medieval',
 'On the Line',
 'Smile',
 'Frank and Penelope',
 'Margaux',
 'Corrective Measures',
 'The Soccer Football Movie',
 "Tom and Jerry Snowman's Land",
 'Slumberland',
 'Enola Holmes 2',
 'The Woman King']

In [198]:
def get_movies_rating(doc):

    desc_selector = 'user_score_chart'
    movies_rating_tags = doc.find_all('div', {'class': desc_selector})
    movies_rating = []
    # Loop through the webpage to get the ratings of all the movies in the page
    for tag in movies_rating_tags:
        movies_rating.append(tag.attrs['data-percent'])
    return movies_rating
get_movies_rating(doc)

['73.0',
 '68.0',
 '53.0',
 '38.0',
 '74.0',
 '75.0',
 '69.0',
 '68.0',
 '74.0',
 '72.0',
 '65.0',
 '68.0',
 '75.0',
 '68.0',
 '49.0',
 '70',
 '80',
 '80',
 '77.0',
 '78.0']

In [199]:
def get_movies_urls(doc):
    """
    Function to extract the movie links from HTML source code using BeautifulSoup. 
    """
    movies_urls = []
    base_url = 'https://www.themoviedb.org'
    movies_names_tags = doc.find_all('h2')[4:]  #starting from the 5th line
    # Loop through the webpage to get the URL of each movie
    for tag in movies_names_tags:
        movies_urls.append(base_url + tag.a['href'])
    return movies_urls
movies_url = get_movies_urls(doc)

In [200]:
def get_detailed_movie_page(movies_url):
    """
    Function to read the HTML source code using BeautifulSoup.
    """

    movies_doc = get_movies_page(movies_url)
    return movies_doc
doc1 =get_detailed_movie_page(movies_url[1])

In [203]:
def get_movies_info(doc):
    """
    Function to get the movie informations - 
    release date, genre, runtime and director.
    """
    div1_tags = doc.find('div', class_ = 'facts')
    release_date = div1_tags.text.split()[1]
    genre = div1_tags.text.split()[3:-2]
    runtime = div1_tags.text.split()[-2:]
    
    div2_tags = doc.find_all('div', {'class':'scroller_wrap should_fade is_fading'})
    director = 'None' #div2_tags[0].text.strip().partition("\n")[0]
    
    return release_date, genre, runtime, director
get_movies_info(doc1)

('11/15/2022',
 ['Fantasy,', 'Action,', 'Comedy,', 'Crime'],
 ['1h', '42m'],
 'None')

In [204]:
def get_all_movies_details(urls):

    genres = []
    release_dates = []
    runtimes = []
    directors = []
    
    # Loop through all the urls of the the movies 
    for url in urls:
        movie_doc = get_movies_page(url)
        # get_movies_info returns release_date, genre, runtime, director.
        release_date, genre, runtime, director = get_movies_info(movie_doc)
        # Convert the genre list to string on ` `. 
        genres.append(" ".join(genre))
        release_dates.append(release_date)
        runtimes.append(" ".join(runtime))
        directors.append(director)
        
    return genres, release_dates, runtimes, directors

In [205]:
def scrape_movies():
    """
    Function to download web page using `requests` and
    to extract the HTML source code using BeautifulSoup.
    """
    # Let's get the popular movies listing from the TMdb website
    page_count = 1 # Initializing the movie page count to 1
    # Define lists for all the movie attributes
    all_names = []
    all_ratings = []
    all_genres = []
    all_release_dates = []
    all_runtimes = []
    all_directors = []
    all_urls = []
    
    while page_count < 9: # Looping for 8 pages of the TMdb web page
        movies_url = "https://www.themoviedb.org/movie?page=%d" %(page_count)
        # Access the webpage using `requests`
        needed_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}
        response = requests.get(movies_url, headers = needed_headers )
        if response.status_code != 200:
            raise Exception('Failed to load page {}'.format(movies_url))
        # Parse the `response' text using BeautifulSoup
        doc = BeautifulSoup(response.text, 'html.parser')
        
        urls = get_movies_urls(doc)
        genres, release_dates, runtimes, directors = get_all_movies_details(urls)
        
        # Append each movie attribute to respective lists
        all_names += get_movies_names(doc)
        all_ratings += get_movies_rating(doc)
        all_genres += genres
        all_release_dates += release_dates
        all_runtimes += runtimes
        all_directors += directors
        all_urls += urls 
        page_count += 1

        # Defining a dictionary to store the movie informations
    movies_dict = {
        'name': all_names,
        'rating': all_ratings,
        'genre': all_genres,
        'release_date': all_release_dates,
        'runtime': all_runtimes,
        'director': all_directors,
        'url': all_urls
    }
    return pd.DataFrame(movies_dict)

In [206]:
scrape_movies()

,name,rating,genre,release_date,runtime,director,url
0,Black Adam,73.0,"Action, Fantasy, Science Fiction",10/21/2022,2h 5m,None,https://www.themoviedb.org/movie/436270
1,R.I.P.D. 2: Rise of the Damned,68.0,"Fantasy, Action, Comedy, Crime",11/15/2022,1h 42m,None,https://www.themoviedb.org/movie/1013860
2,Paradise City,53.0,"Crime, Action, Thriller",11/17/2022,1h 33m,None,https://www.themoviedb.org/movie/829799
3,Hex,38.0,"Action, Horror, Thriller",11/01/2022,1h 28m,None,https://www.themoviedb.org/movie/988233
4,Disenchanted,74.0,"Comedy, Family, Fantasy",11/18/2022,1h 59m,None,https://www.themoviedb.org/movie/338958
...,...,...,...,...,...,...,...
155,Harry Potter and the Philosopher's Stone,79.0,"Adventure, Fantasy",11/16/2001,2h 32m,None,https://www.themoviedb.org/movie/671
156,Wire Room,68.0,"Action, Crime, Thriller",09/02/2022,1h 37m,None,https://www.themoviedb.org/movie/921360
157,Your Boyfriend Is Mine,68.0,"Thriller, TV Movie, Action",03/19/2022,1h 25m,None,https://www.themoviedb.org/movie/951368
158,Uncharted,71.0,"Action, Adventure",02/18/2022,1h 56m,None,https://www.themoviedb.org/movie/335787
